In [ ]:
knitr::opts_chunk$set(echo = TRUE)

## Description - EDA Report 2

# Title and Introduction

Analysts - Tigris Mendez and Dan Tatulescu

We would like to know how construction affects traffic incident rates. We use construction permits as a proxy for the actual number of construction sites in a zip code. Our motivation is to show high risk construction projects, which may deserve a greater degree of road safety preparation. [This study](https://www.sciencedirect.com/science/article/pii/S235214652100819X), by Mangones, et al. (2021), has shown that construction projects can increase traffic incidents when there is use of excavation of more than half a meter. We wish to localize this experiment to Austin and understand how the different types of construction projects affect traffic incident rates.

Name - [Issued Construction Permits](https://data.austintexas.gov/Building-and-Development/Issued-Construction-Permits/3syk-w9eu/about_data)

Description - This data set contains "Building, Electrical, Mechanical, and Plumbing Permits and Driveway/Sidewalk Permits issued by the City of Austin. Includes relevant details such as issue date, location, council district, expiration date, description of work, square footage, valuation, and units." (City of Austin open data portal)

Rows & Columns - There are 897369 rows and 68 columns.

Unique Rows - A single row represents a construction permit.

**Main Variables of Interest - Construction Permits**

-   **Permit.Type.Desc** -\> Description of the Permit Type
-   **Permit.Class** -\> "Sub Type of the permit", it will be re-categorized as Residential or Commercial permits.
-   **Issued.Date** -\> Date on which the permit was issued
-   **Status.Current** -\> Current status of permit
-   **Number.Of.Floors** -\> How many floors property has
-   **Original.Zip** -\> Zip code of the property associated with the permit

### Dataset 2

Name - [Real-Time Incidents](https://data.austintexas.gov/Transportation-and-Mobility/Real-Time-Traffic-Incident-Reports/dx9v-zd7x/about_data)

Description - "This data set contains various traffic incidents from the Austin-Travis County traffic reports collected from the various Public Safety agencies through a data feed from the Combined Transportation, Emergency, and Communications Center (CTECC)." (City of Austin open data portal)

Rows & Columns - There are 356150 rows and 22 columns.

Unique Rows - A single row represents a traffic incident.

**Main Variables of Interest - Traffic Incidents:**

-   **Published_Date** -\> The date the report was published
-   **Issue_Reported** -\> The reported issue, based on the selection by reporting agency
-   **Zip code** -\> Processed from lat/lon using ArcGIS reverse-geocoding

## Expecations

**Trends and Relationships**

We expect that Commercial Projects cause an increase in Traffic Incident rates. We expect that there will be more traffic incidents during the months of mid spring when there is more rain. We also expect that there will be a difference in Traffic Incidents rates based upon the proximity of zip codes to the Austin city center.

**Research Question - Dan**

Is there a difference in impact of Residential or Commercial construction projects on Traffic Incidents? We will eventually determine this causal relationship through the rejection of the null hypothesis which states that the impact of Commercial Permits and Residential Permits on Traffic Incidents are equivalent.

**Research Question - Tigris**

How do spatial concentrations of Traffic Incidents change throughout the year?

## Install & Call Libraries

In [ ]:
#install.packages("ggmap")
#install.packages
#install.packages("tidygeocoder")
#install.packages("ggplot2")
#install.packages("sf")
#install.packages("tigris")
library("sf")
library("tigris")
library("dplyr")
library("ggplot2")
library("ggmap")
library("tidyverse")
library("zipcodeR")
library("tidygeocoder")

# Methods

## Inputting Data

In [ ]:
# This code reads the Construction Permit data set into a R dataframe
construction_permits <- read.csv("~/My Documents/UT Course Files/UT Fall 2024/SDS 322E/SDS Project Files/Original Data/Issued_Construction_Permits_20241006.csv", header=TRUE)

head(construction_permits)
as.data.frame(colnames(construction_permits))

In [ ]:
dim(construction_permits)

**Note - ArcGIS Reverse Geocoding Process**

In the Traffic Incidents data set there was not a column for Zip codes. Because there was available data for latitude and longitude, we decided to reverse-geocode the lat/lon values in a column of zip codes. We initially were attempting to utilize Maps API services to perform this task but it was cost prohibitive. We opted to use a Geographic Information System software named ArcGIS to produce the Zipcode data leveraging a TIGER/Line Shapefile provided by the US Census Bureau to reverse geocode the lat/lon figures through ArcGIS's geoferencing capabilities.

In [ ]:
# This code reads the Traffic Incidents data set into a R dataframe
traffic_indcidents <- read.csv("~/My Documents/UT Course Files/UT Fall 2024/SDS 322E/SDS Project Files/Original Data/joined_traffic_zipcode_csv.csv")

head(traffic_indcidents)
as.data.frame(colnames(traffic_indcidents))

In [ ]:
dim(traffic_indcidents)

## Cleaning Data

### Removing unneeded columns

In [ ]:
# Selecting only the useful columns for Traffic Indcidents Dataset
ti_df = traffic_indcidents[,c('Published_Date',
                              'Issue_Reported',
                              'ZCTA5CE20')]

ti_df |>
  count(Issue_Reported)

# renaming the zipcode column
ti_df <- ti_df |>
  rename(zipcode = ZCTA5CE20)

In [ ]:
# Selecting only the useful columns for Construction permits
cp_df = construction_permits[,c('Permit.Class',
                                'Permit.Type.Desc',
                                'Issued.Date',
                                'Status.Current',
                                'Number.Of.Floors',
                                'Original.Zip')]

unique(cp_df$Permit.Type.Desc)

In [ ]:
# if starting character equals R, we isolate it to represent Residential
# if starting character equals C, we isolate it to represent Commercial
cp_df$Permit.Class.Simple <- ifelse(cp_df$Permit.Class == "", "",
                                    substr(cp_df$Permit.Class, 1, 1))

cp_df$Permit.Class.Simple <- ifelse(cp_df$Permit.Class.Simple == "S", NA,
                                    cp_df$Permit.Class.Simple)

cp_df$Permit.Class.Simple <- ifelse(cp_df$Permit.Class.Simple == "", NA,
                                    cp_df$Permit.Class.Simple)

unique(cp_df$Permit.Type.Desc)
head(cp_df)
unique(cp_df$Status.Current)
dim(cp_df)

### Filtering Status.Current for only Final, Active,

In [ ]:
# Find the number of each Status among the cp_df dataset
cp_df |>
  count(Status.Current)

**After counting the number of Status occurrences, we see that 'Final', 'Active' and 'Expired' are the most common status among the permits. We omit 'Expired' and the rest of the categories as they do not have a large impact on the number of traffic incidents. (Note: Some permits may have status that implies that a new permit was created. Because of the involved ambiguity, and the fact that these number of these permits is low, we do not include them in the data set from here on.)**

In [ ]:
# Filtering only for permits with Status of 'Active' or 'Final'
cp_df <- cp_df |>
  filter(Status.Current %in% c('Final', 'Active'))

cp_df |>
  count(Status.Current)

### NA Values

Count NA Values

In [ ]:
# counts the number of rows with missing values
missing_rows_count_ti <- sum(!complete.cases(ti_df))
missing_rows_count_cp <- sum(!complete.cases(cp_df))


sprintf("There are % s rows with missing values in the traffic incidents dataset.", missing_rows_count_ti)
sprintf("There are % s rows with missing values in the construction permits dataset.", missing_rows_count_cp)

Drop NA Values

In [ ]:
# dropping NA values from the Traffic Incidents data set & saving the data frame
ti_df_noNA <- na.omit(ti_df)

In [ ]:
# assign rows with description type of "Driveway / Sidewalks" to 0
cp_df$Number.Of.Floors <- ifelse(is.na(cp_df$Number.Of.Floors) & cp_df$Permit.Type.Desc == "Driveway / Sidewalks", 0, cp_df$Number.Of.Floors)

# Drop Number.of.floors above 80 (The tallest building in Austin is 75-floors)
cp_df <- cp_df |>
  filter(Number.Of.Floors <= 75)

# dropping NA values from the Construction Permits data set & saving the data frame
cp_df_noNA <- na.omit(cp_df)

Rows and Columns of resulting data set after cleaning

In [ ]:
# Rows and Columns of the cleaned Traffic Incident data set
dim(ti_df_noNA)
# Rows and Columns of the cleaned Construction Permit data set
dim(cp_df_noNA)

**In the Traffic Incident data set we started with 356,150 rows and 22 columns.** **In the Construction Permits data set we started with 897,369 rows and 68 columns.** **In the Traffic Incident data set we ended with 355,333 and 3 columns.** **In the Construction Permits data set we ended with 737,781 and 7 columns.**

We selected certain rows based on their relevance to our area of investigation, which reduced the number of columns. We utilized ArcGIS Pro to create a zipcode column for traffic incidents based on the existing latitude and longitude columns for each row.

We also dropped any rows that contained missing values across the columns of interest. And prior to that, we converted the number of floors for rows with the 'Driveway/Sidewalk' permit type description and missing floor values to be 0 as we can safely assume that sidewalks and driveways are on at ground level.

The resulting data tidy in each data set as each column represents a characteristic, each row represents an individual permit or incident, and each cell represents an individual value.

To match the data sets we need to group by zip codes to relate the two datasets togehter. We do this by using the groupby() method.

# Results

## Research Question 1 - EDA ; Visualization #1 & #2

Is there a difference in impact of Residential or Commercial construction projects on Traffic Incidents?

In [ ]:
# find the number of zip codes
zip_code_counts <- cp_df_noNA |>
  count(Original.Zip)

Zip Code Counts - Summary Statistics

In [ ]:

zip_count_r <- cp_df_noNA[cp_df_noNA$Permit.Class.Simple == "R",] |>
  count(Original.Zip)

zip_count_c <- cp_df_noNA[cp_df_noNA$Permit.Class.Simple == "C",] |>
  count(Original.Zip)

# plot distribution of residential permits and commercial permits across the zipcode
ggplot(zip_count_r, aes(x = n)) +
  geom_histogram(fill = "skyblue", binwidth = 750, center = 375) +
  labs(title = "Viz. 1: Number of Zipcodes with X permits (residential).",
       y = "Number of Zipcodes", 
       x = 'Residential Construction Permits')

summary(zip_count_r)

**There are more zipcodes with fewer residential construction permits. The data is also positively skewed. The mean number of permits per zipcode is 9,876 whilst the median is 7,043.**

In [ ]:
ggplot(zip_count_c, aes(x = n)) +
  geom_histogram(fill = "red", binwidth = 750, center = 375) +
  xlim(0, 40000) +
  labs(title = "Viz. 2: Number of Zip Codes with X level of Construction Permits (commercial).",
       y = "Number of Zipcodes", 
       x = 'Commercial Construction Permits')

summary(zip_count_c)

``` {r}
zip_count_c_noOutlier <- zip_count_c |> 
  filter(n < 20000)

ggplot(zip_count_c_noOutlier, aes(x = n)) +
  geom_histogram(fill = "red", binwidth = 750, center = 375) +
  xlim(0, 40000) +
  labs(title = "Viz. 2: Number of Zip Codes with X level of Construction Permits (commercial).",
       y = "Number of Zipcodes", 
       x = 'Commercial Construction Permits')

summary(zip_count_c_noOutlier)
```

**There are more zipcodes with fewer commercial construction permits. The data is also positively skewed. The mean number of permits per zipcode is 4,122 whilst the median is 2,576.**

In [ ]:
zip_code_counts_TI <- ti_df_noNA |> count(zipcode)

summary(zip_code_counts_TI)

permit_join <- full_join(zip_count_c_noOutlier, zip_count_r, by = c("Original.Zip"))

permit_join <- permit_join |> rename(C_count = n.x, R_count = n.y)

total_join <- full_join(permit_join, zip_code_counts_TI, by = c("Original.Zip" = "zipcode")) |> rename(TI_count = n)

total_join |>
  drop_na() |>
  ggplot() +
  geom_point(aes(x = R_count, y = TI_count), color = "skyblue") +
  labs(title = "Viz. 3: # of Traffic Incidents vs. # of Construction Permits", y = "Number of Traffic Incidents", x = 'Number of Residential Construction Permits')

total_join |>
  drop_na() |>
  ggplot() +
  geom_point(aes(x = C_count, y = TI_count), color = "red") +
  labs(title = "Viz. 4: # of Traffic Incidents vs. # of Comm. Construction Permits", y = "Number of Traffic Incidents", x = 'Number of Commercial Construction Permits')

summary(total_join)

**All rows with zip codes that are not in the traffic incident data set is removed. There is one zipcode missing from the commercial permits that is in the residential permits. There is 53 points in the residential scatter plot. There are 52 points in the commercial data set where each point is a zipcode. There appears to be a positive correlation between traffic incidents and both residential and commercial construction permit numbers. However, the correlation with residential construction permits is much stronger. We already discussed the commercial and resident permit statistics, but here, let us bring your attention to the traffic incidents. The mean number of traffic incidents per zipcode is 4,281 with a median of 1,917, again indicating positive skew.**

## Research Question 2 - EDA ; Visualization #3 & #4

How do spatial concentrations of Traffic Incidents change throughout the year?

### Visualization 1

In [ ]:
ti_df_formatted <- ti_df_noNA

In [ ]:
ti_df_formatted$Published_Date <- as.POSIXct(ti_df_formatted$Published_Date, format = "%m/%d/%Y %H:%M:%OS")

# Converting 'Issued.Date' to Date to ensure correct format
#ti_df_noNA$Published_Date <- as.Date(ti_df_noNA$Published_Date)


# Extracting the year from 'Issued.Date'
ti_df_formatted$Year <- as.integer(format(ti_df_formatted$Published_Date, "%Y"))

ti_df_formatted <- na.omit(ti_df_formatted) |>
  filter(Year < 2024)

incidents_summary <- ti_df_formatted |>
  group_by(`zipcode`) |>
    summarize(incidents_count = n())

# showing the number of incidents per year
years_count <- ti_df_formatted |>
  count(Year)

years_count

summary(ti_df_formatted)


In [ ]:
# Show the traffic incidents per year
ti_df_formatted |>
  ggplot(aes(x = Year)) +
  geom_histogram(binwidth = 1, center = 0, color = "darkgreen", fill = "lightgreen") +
  scale_x_continuous(breaks = seq(2017, 2023, by = 1), labels = seq(2017, 2023, by = 1)) +
  labs(title = "Viz. 5: Traffic Incidents per Year")

**There are 26 incidents in the year of 2024, we will remove these outliers since there are not very many.**

In [ ]:
# Fetch ZIP code shapefiles for Texas
zipcodes <- zctas(year = 2010, state = "TX") |>
  select(ZCTA5CE10, geometry) |>
  mutate(ZCTA5CE10 = as.numeric(ZCTA5CE10))

ti_df_17_20 <- ti_df_formatted |>
  filter(Year <= 2020) |>
  group_by(`zipcode`) |>
  summarize(incidents_count = n())

summary(ti_df_17_20)

ti_df_21_23 <- ti_df_formatted |>
  filter(Year > 2020) |>
  group_by(`zipcode`) |>
  summarize(incidents_count = n())

summary(ti_df_21_23)

In [ ]:

join_17_20 <- ti_df_17_20 |>
  inner_join(zipcodes, by = c("zipcode" = "ZCTA5CE10"))

ti_df_17_20 |>
  # Finds the elements that exists in both the shape file and the coyotes_summary data set
  inner_join(zipcodes, by = c("zipcode" = "ZCTA5CE10")) |>
  # ggplot sets the charting library
  ggplot() +
  # 
  geom_sf(aes(geometry = geometry, fill = incidents_count), color = "black") +
  
  labs(title = "Viz. 6: Traffic Incidents between 2017 and 2020", x = "Longitude", y = "Latitude")

ti_df_21_23 |>
  # Finds the elements that exists in both the shape file and the coyotes_summary data set
  inner_join(zipcodes, by = c("zipcode" = "ZCTA5CE10")) |>
  # ggplot sets the charting library
  ggplot() +
  # 
  geom_sf(aes(geometry = geometry, fill = incidents_count), color = "black") +
  
  labs(title = "Viz. 7: Traffic Incidents between 2021 and 2023", x = "Longitude", y = "Latitude")

**These visualizations agree with our hypothesis that traffic incidents would be more focused in the city center. There are more zipcodes in the dataset with the years of 2017 through 2020. Between the years of 2017 and 2020 the median and mean number of traffic incidents is 1,245 and 2,501, respectively. Between the years of 2021 and 2024 the median and mean number of traffic incidents is 1,123 and 1,963.7, respectively.**

### Visualization 2

How do trends in traffic incidents changes throughout the year (difference by month)

In [ ]:
# Extracting the month from 'Issued.Date'
ti_df_formatted$Month <- as.integer(format(ti_df_formatted$Published_Date, "%m"))

ti_df_formatted |>
  ggplot(aes(x = Month)) +
  geom_histogram(binwidth = 1, center = 0, color = "darkgreen", fill = "lightgreen") +
  scale_x_continuous(breaks = seq(1, 12, by = 1), labels = seq(1, 12, by = 1)) +
  labs(title = "Viz. 8: Traffic Incident Counts Across Each Month")

# displays the summary statistics for the 
# number of traffic incidents during each month
ti_df_formatted |>
  count(Month) |>
  summary()

**For the number of traffic incidents across the months, there is a spike in accidents during October that decreases during January. This indicates an increase in traffic incidents in the winter months. The median number of traffic incidents across the months is 28,895 while the mean is 29,604. There is a Min of 27,278 and a Max of 35,308.**

In [ ]:
ti_df_formatted |>
  filter(Year <= 2020) |>
  ggplot(aes(x = Month)) +
  geom_histogram(binwidth = 1, center = 0, color = "blue", fill = "skyblue") +
  scale_x_continuous(breaks = seq(1, 12, by = 1), labels = seq(1, 12, by = 1)) +
  labs(title = "Viz. 9: Traffic Incident Counts From 2017 To 2020")

ti_df_formatted |>
  filter(Year <= 2020) |>
  count(Month) |>
  summary()

**When we separate the the month against traffic incident count into two different segments based on year, we find that between the years of 2017 and 2020 the trend is similar to the total trend. There is a slightly more pronounced increase in traffic incidents in the winter months. The median and mean number of traffic incidents are 15,628 and 16,676, respectively with a Min of 14,650 and a Max 21,184.**

In [ ]:
ti_df_formatted |>
  filter(Year > 2020) |>
  ggplot(aes(x = Month)) +
  geom_histogram(binwidth = 1, center = 0, color = "red", fill = "orange") +
  scale_x_continuous(breaks = seq(1, 12, by = 1), labels = seq(1, 12, by = 1)) +
  labs(title = "Viz. 10: Traffic Incident Counts From 2021 through 2023")

ti_df_formatted |>
  filter(Year > 2020) |>
  count(Month) |>
  summary()

**Between the years of 2021 and 2023 the median and mean number of traffic incidents are 12,793 and 12,928, respectively with a Min of 11,361 and a Max 14,150. This shows that the "winter effect" is not as present in years closer to the most recent years in the data set.**

# SDS 322E Project: Part 2

## Classification and Prediction

## Description - Classification and Prediction Report

**We want to predict the number of Traffic Incidents. Our predictive variables will be:**
**- Permit.Class.Simple**
**- Permit.Type.Desc**
**- Status.Current**
**- Number.Of.Floors**

**Our outcome variable is TI_count, the number of traffic incidents in a Zip code**

**We will use both Linear Regression and KNN to predict Traffic Incident Counts as it is a continuous numeric variable.**

In [ ]:
# Creating a new data frame for predictive analysis
TI_Counts <- ti_df_formatted |>
  group_by(`zipcode`) |>
  summarize(incidents_count = n())

# Assuming cp_df_noNA is your main dataframe
encoded_counts <- cp_df_noNA |>
  mutate(
    is_R = as.integer(Permit.Class.Simple == "R"),
    is_C = as.integer(Permit.Class.Simple == "C"),
    is_Building = as.integer(Permit.Type.Desc == "Building Permit"),
    is_Sidewalks = as.integer(Permit.Type.Desc == "Driveway / Sidewalks"),
    is_Electric = as.integer(Permit.Type.Desc == "Electrical Permit"),
    is_Mechanical = as.integer(Permit.Type.Desc == "Mechanical Permit"),
    is_Plumbing = as.integer(Permit.Type.Desc == "Plumbing Permit"),
    is_Active = as.integer(Status.Current == "Active"),
    is_Final = as.integer(Status.Current == "Final")
  ) |>
  group_by(Original.Zip) |>
  summarize(
    R_count = sum(is_R),
    C_count = sum(is_C),
    Building_count = sum(is_Building),
    Sidewalks_count = sum(is_Sidewalks),
    Electric_count = sum(is_Electric),
    Mechanical_count = sum(is_Mechanical),
    Plumbing_count = sum(is_Plumbing),
    Active_count = sum(is_Active),
    Final_count = sum(is_Final),
    Mean_Number_Of_Floors = round(mean(Number.Of.Floors, na.rm = TRUE), digits = 2)
  )

# Join with TI_Counts
final_join <- inner_join(encoded_counts, TI_Counts, by = c("Original.Zip" = "zipcode"))
#------------------------

final_join

# Model 1 - Linear Regression

To asses the performance of the models, We will perform cross-validation. Specifically, we will perform a 10-fold cross-validation.

``` {r}
final_join <- final_join |>
    filter(C_count < 20000)


final_join <- final_join |>
    filter(Mean_Number_Of_Floors < 5)

max(final_join$C_count)
max(final_join$Mean_Number_Of_Floors)
```

In [ ]:
# Implement a Linear Regression Model
fit_lin <- lm(incidents_count ~ R_count + C_count + Mean_Number_Of_Floors, data = final_join)

summary(fit_lin)


``` {r}
final_join |> 
  # Calculate predicted values
  mutate(predicted = predict(fit_lin)) |> 
  # Use a ggplot to represent the relationship
  ggplot(aes(x = R_count + C_count + Mean_Number_Of_Floors, y = incidents_count)) +
  # Add the linear model
  geom_smooth(method = "lm", se = FALSE, color = "steelblue", size = 2) + 
  # Add residuals = vertical segments from observations to predicted
  geom_segment(aes(xend = R_count + C_count + Mean_Number_Of_Floors, yend = predicted), alpha = .5, color = "red") +
  # Display the observed data
  geom_point(size = 4) +
  # Display the predicted (on top of the line)
  geom_point(aes(y = predicted), size = 4, color = "orange") +
  labs(x = "R_count + C_count + Mean_Number_Of_Floors",
       y = "Count of incidents",
       title = "Linear regression model with residuals")
```
**In the graphic above, the orange dots represent predicted values while the black dots represent the actual values. The red line represent the difference between the two values, known as the residual values.**



In [ ]:

final_join2 <- final_join |>
  mutate(predictions = predict(fit_lin, final_join)) |>
  select(incidents_count, R_count, C_count, Mean_Number_Of_Floors, predictions)

# Scatter plot of actual vs predicted incidents count
ggplot(final_join2, aes(x = incidents_count, y = predictions)) +
  geom_point(color = "blue", size = 3) + # Points for actual vs predicted
  geom_abline(intercept = 0, slope = 1, color = "red", linetype = "dashed") + # Ideal fit line
  labs(
    title = "Actual vs Predicted Incidents Count",
    x = "Actual Incidents Count",
    y = "Predicted Incidents Count"
  )

cor(final_join2$incidents_count, final_join2$predictions)


**The actual and predicted values have a correlation 79.59%**

In [ ]:
RMSE = sqrt(mean(resid(fit_lin)^2))
RMSE

**Adding predicted and residuals to the data frame.**

``` {r}
final_join |> 
  # First add predicted values based on model
  mutate(predicted = predict(fit_lin)) |> 
  # Calculate residuals = observed - predicted
  mutate(residuals = incidents_count - predicted) |>
  # Only display variables of interest
  select(R_count, C_count, Mean_Number_Of_Floors, incidents_count, predicted, residuals)
```

# Cross-validation - Linear Regression

``` {r}
# Choose number of folds
k = 5 

# To have the same random sample, use set.seed
set.seed(322)

# Randomly order rows in the dataset
data <- final_join[sample(nrow(final_join)), ] 

# Create k folds from the dataset
folds <- cut(seq(1:nrow(data)), breaks = k, labels = FALSE)
```

``` {r}
# Initialize a vector to keep track of the performance for each k-fold
perf_k <- NULL

# Use a for-loop to get performance for each k-fold
for(i in 1:k){
  # Split data into train and test data
  train_not_i <- data[folds != i, ] # train data = all observations except in fold i
  test_i <- data[folds == i, ]  # test data = observations in fold i
  
  # Train model on train data (all but fold i)
  # CHANGE: what model/predictors should be included
  train_model <- lm(incidents_count ~ R_count + C_count + Mean_Number_Of_Floors, data = train_not_i)
      
  
  # Performance listed for each test data = fold i
  # CHANGE: how the performance is calculated
  perf_k[i] <- sqrt(mean((test_i$incidents_count - predict(train_model, test_i))^2))
}

# Performance for each fold 
perf_k

# Average performance over all k folds
mean(perf_k)
sd(perf_k)
```
**Cross validation of the Linear Regression model shows us that the average RMSE, the difference between the actual and predicted values, is 3351.139. Since the Standard Deviation of the RMSE across folds is particularly large at 1,187.919, the prediction model is not particularly consistent across the different folds.** 

# Model 2 - K-nearest neighbors (KNN)

In [ ]:
#install.packages("caret")
library(caret)

fit_knnreg <- knnreg(incidents_count ~ R_count + C_count + Mean_Number_Of_Floors,
                     data = final_join,
                     k = 5) # Number of Neighbors

final_join1 <- final_join |>
  mutate(predictions = predict(fit_knnreg, final_join)) |>
  select(incidents_count, R_count, C_count, Mean_Number_Of_Floors, predictions)

sqrt(mean((final_join$incidents_count - predict(fit_knnreg, final_join))^2))

final_join1 <- final_join1 |> 
  mutate(prediction_group = cut(predictions, breaks = 5, labels = FALSE))

# Scatter plot to visualize feature space with color-coded predictions
ggplot(final_join1, aes(x = R_count, y = incidents_count, color = as.factor(prediction_group))) +
  geom_point(size = 3) +
  labs(
    title = "KNN Predicted Grouping in Feature Space",
    x = "R_count",
    y = "incidents_count",
    color = "Prediction Group"
  )

# Scatter plot to visualize feature space with color-coded predictions
ggplot(final_join1, aes(x = C_count, y = incidents_count, color = as.factor(prediction_group))) +
  geom_point(size = 3) +
  labs(
    title = "KNN Predicted Grouping in Feature Space",
    x = "C_count",
    y = "incidents_count",
    color = "Prediction Group"
  )

# Scatter plot to visualize feature space with color-coded predictions
ggplot(final_join1, aes(x = Mean_Number_Of_Floors, y = incidents_count, color = as.factor(prediction_group))) +
  geom_point(size = 3) +
  labs(
    title = "KNN Predicted Grouping in Feature Space",
    x = "Mean_Number_Of_Floors",
    y = "incidents_count",
    color = "Prediction Group"
  )


``` {r}


# Scatter plot of actual vs predicted incidents count
ggplot(final_join1, aes(x = incidents_count, y = predictions)) +
  geom_point(color = "blue", size = 3) + # Points for actual vs predicted
  geom_abline(intercept = 0, slope = 1, color = "red", linetype = "dashed") + # Ideal fit line
  labs(
    title = "Actual vs Predicted Incidents Count",
    x = "Actual Incidents Count",
    y = "Predicted Incidents Count"
  )

cor(final_join1$incidents_count, final_join1$predictions)


final_join1 |> 
  # Calculate predicted values
  mutate(predicted = predict(fit_knnreg, final_join)) |> 
  # Use a ggplot to represent the relationship
  ggplot(aes(x = R_count + C_count + Mean_Number_Of_Floors, y = incidents_count)) +
  # Add the linear model
  geom_smooth(method = "lm", se = FALSE, color = "steelblue", size = 2) + 
  # Add residuals = vertical segments from observations to predicted
  geom_segment(aes(xend = R_count + C_count + Mean_Number_Of_Floors, yend = predicted), alpha = .5, color = "red") +
  # Display the observed data
  geom_point(size = 4) +
  # Display the predicted (on top of the line)
  geom_point(aes(y = predicted), size = 4, color = "orange") +
  labs(x = "R_count + C_count + Mean_Number_Of_Floors",
       y = "Count of incidents",
       title = "Linear regression model with residuals")


```
**In the graphic above, the orange dots represent predicted values while the black dots represent the actual values. The red line represent the difference between the two values, known as the residual values. The actual and predicted values have a correlation 85.71%, which is higher than that from the linear regression, though the RMSE is also higher (seen below).**

# Cross-validation - K-nearest neighbors (KNN)

In [ ]:
# Choose number of folds
k = 5 

# To have the same random sample, use set.seed
set.seed(322)

# Randomly order rows in the dataset
data <- final_join[sample(nrow(final_join)), ] 

# Create k folds from the dataset
folds <- cut(seq(1:nrow(data)), breaks = k, labels = FALSE)



# Initialize a vector to keep track of the performance for each k-fold
perf_k <- NULL

# Use a for-loop to get performance for each k-fold
for(i in 1:k){
  # Split data into train and test data
  train_not_i <- data[folds != i, ] # train data = all observations except in fold i
  test_i <- data[folds == i, ]  # test data = observations in fold i
  
  # Train model on train data (all but fold i)
  # CHANGE: what model/predictors should be included
  train_model <- knnreg(incidents_count ~ R_count + C_count + Mean_Number_Of_Floors,
                     data = train_not_i,
                     k = 5)
  
  # Performance listed for each test data = fold i
  # CHANGE: how the performance is calculated
  perf_k[i] <- sqrt(mean((test_i$incidents_count - predict(train_model, test_i))^2))
}

# Performance for each fold 
perf_k

# Average performance over all k folds
mean(perf_k)
sd(perf_k)

**Cross validation of the KNN model shows us that the average RMSE, the difference between the actual and predicted values, is 3787.428. Since the Standard Deviation of the RMSE across folds is particularly large at 1,217.553, the prediction model is not particularly consistent across the different folds, though it is still comparable to the linear regression, especially when taken in combination with the higher correlation coefficient.** 

# Discussion

There is indeed a difference in impact between Residential construction project and Commercial construction projects on Traffic Incidents. Specifically looking at the scatterplots (Vis. 3 and 4), we see that there is a stronger positive correlation between the number of Commercial construction permits and the number of Traffic Incidents than between the number of Residential construction projects and the number of Traffic Incidents, though as we expected, both seem to show positive correlations.

The visualization match our expectations as shown in Visualization 6 & 7 as the number traffic incidents increase per zipcode as they are closer in proximity to the city center. It is important to note that the zipcodes that are closer to the city center are smaller in area and higher in density. Visualizations 9 & 10 show that in earlier years (Vis. 9) the number of traffic incidents increased in the winter months while in recent years (Vis. 10) the traffic incident counts across the month are more uniform. We would like to note that shape file that we used was for the year of 2010 while the data is for the years of 2017 through 2023. If these ZIP codes were changed in their spatial construction, the results of this study may differ, though we do expect these differences to be minor.

The biggest takeaways from this EDA Study would be that 1) In past years, there was an increase in traffic incidents during the winter months compared to more recent years, 2) The density of traffic incidents increases in the ZIP codes closer in proximity to the city center, 3) There are generally more zipcodes with fewer permits when looking at either residential or commercial construction permits, and 4) There is a stronger positive correlation between the number of commercial construction permits and the number of traffic incidents than there is between the number of residential construction permits and the number of traffic incidents.

# Reflection, acknowledgements, and references

One of the biggest challenges was understanding how to get the data into the correct dataframes and format to answer our research questions. To accomplish this task we had to understand certain syntax rules in R, particularly considering grouping, counting, and using shapefiles to map geospatial data. We learned how to better clean and visualize our data in order to answer our research questions to a satisfactory degree. Furthermore, we came to realize how important it is to clean the data before beginning our analysis and how to put certain values in useable formats, e.g., dates.

The datasets are linked above, where one can find information about the data owners. We would like to extend our gratitude to our professor, Dr. Layla Guyot, and our UGCA, Vamsi Abena, for their help and guidance with our EDA. Also, thank you to the City of Austin for making this data publicly accessible so that we can carry out this project. We, Tigris and Dan, contributed to equal parts of the project with special individual contributions to our particularly research questions.

# Links and References

[Construction Permit Dataset](https://data.austintexas.gov/Building-and-Development/Issued-Construction-Permits/3syk-w9eu/about_data) [Traffic Incident Dataset](https://data.austintexas.gov/Transportation-and-Mobility/Real-Time-Traffic-Incident-Reports/dx9v-zd7x/about_data) [Crash Rates During Non-Construction and Construction Periods (Mangones et al., 2021)](https://www.sciencedirect.com/science/article/pii/S235214652100819X)